# Querying

We assume that you have [generated a SDK](generation.html) for the `Windmill` model and have a client ready to go.

`pygen` supports quering nodes via edges. Including, handeling circular dependencies. 

For example, if we want to list the all the blades with sensor positoins we can do the followind query:

In [9]:
import warnings
warnings.filterwarnings('ignore')
# This is just to enable improting the generated SDK from the examples folder in the pygen repository
import sys
from tests.constants import REPO_ROOT
sys.path.append(str(REPO_ROOT / "examples" ))

In [10]:
from windmill.client import WindmillClient

In [11]:
client = WindmillClient.from_toml("config.toml")

## Querying along Edges

In [12]:
blades = client.blade(limit=5).sensor_positions(limit=-1).query()

In [13]:
blades[0].model_dump()

{'space': 'windmill-instances',
 'external_id': 'blade:1',
 'version': 1,
 'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
 'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
 'deleted_time': None,
 'is_damaged': False,
 'name': 'A',
 'sensor_positions': [{'space': 'windmill-instances',
   'external_id': 'sensorposition:1',
   'version': 1,
   'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
   'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
   'deleted_time': None,
   'edgewise_bend_mom_crosstalk_corrected': None,
   'edgewise_bend_mom_offset': None,
   'edgewise_bend_mom_offset_crosstalk_corrected': None,
   'edgewisewise_bend_mom': None,
   'flapwise_bend_mom': 'V52-WindTurbine.MxA6',
   'flapwise_bend_mom_crosstalk_corrected': None,
   'flapwise_bend_mom_offset': None,
   'flapwise_bend_mom_offset_crosstalk_corrected': Non

**Call Explaination**
The call `client.blade(limit=5).sensor_positions(limit=-1).query()` requires some extra explaination. 

1. The first call, `.blade(limit=5)`, filters which blades that we want.
2. In the second call, `.sensor_positions(limit=-1)`, we specify that we want all sensor position edges connected to the blades we found in the first call.
3. The last call `.query()` tells `pygen` to execute the query.


We can build arbitrary long queries this way and we always finish with `.query()` to execute the query. For example, we can take all windmills at the 'Hornsea 1' windfarm and include all blades and sensor position connected to these blades.

In [15]:
windmills_at_hornsea_with_blades_and_sensors = client.windmill(windfarm="Hornsea 1").blades(limit=-1).sensor_positions(limit=-1).query()

In [16]:
windmills_at_hornsea_with_blades_and_sensors

,external_id,blades,capacity,metmast,nacelle,name,rotor,windfarm
0,hornsea_1_mill_3,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacelle:1,hornsea_1_mill_3,rotor:1,Hornsea 1
1,hornsea_1_mill_2,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacelle:2,hornsea_1_mill_2,rotor:2,Hornsea 1
2,hornsea_1_mill_1,"[{'space': 'windmill-instances', 'external_id'...",7.0,None,nacelle:3,hornsea_1_mill_1,rotor:3,Hornsea 1


In [17]:
for windmill in windmills_at_hornsea_with_blades_and_sensors:
    print(f"{windmill.name} has {len(windmill.blades)} blades")
    for blade in windmill.blades:
        print(f"{blade.external_id} is connected to {windmill.name} and has sensors: {', '.join([sensor.external_id for sensor in blade.sensor_positions])}")
    print("\n")

hornsea_1_mill_3 has 3 blades
blade:1 is connected to hornsea_1_mill_3 and has sensors: sensorposition:1, sensorposition:2, sensorposition:3, sensorposition:4, sensorposition:5, sensorposition:6
blade:2 is connected to hornsea_1_mill_3 and has sensors: sensorposition:7, sensorposition:8, sensorposition:9, sensorposition:10, sensorposition:11, sensorposition:12
blade:3 is connected to hornsea_1_mill_3 and has sensors: sensorposition:13, sensorposition:14, sensorposition:15, sensorposition:16, sensorposition:17, sensorposition:18


hornsea_1_mill_2 has 3 blades
blade:4 is connected to hornsea_1_mill_2 and has sensors: sensorposition:19, sensorposition:20, sensorposition:21, sensorposition:22, sensorposition:23, sensorposition:24
blade:5 is connected to hornsea_1_mill_2 and has sensors: sensorposition:25, sensorposition:26, sensorposition:27, sensorposition:28, sensorposition:29, sensorposition:30
blade:6 is connected to hornsea_1_mill_2 and has sensors: sensorposition:31, sensorposition:

We can dig into the nested structure of one of them by dumping it do a dictinary.

In [18]:
windmills_at_hornsea_with_blades_and_sensors[1].model_dump()

{'space': 'windmill-instances',
 'external_id': 'hornsea_1_mill_2',
 'version': 1,
 'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
 'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
 'deleted_time': None,
 'blades': [{'space': 'windmill-instances',
   'external_id': 'blade:4',
   'version': 1,
   'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
   'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
   'deleted_time': None,
   'is_damaged': True,
   'name': 'B',
   'sensor_positions': [{'space': 'windmill-instances',
     'external_id': 'sensorposition:19',
     'version': 1,
     'last_updated_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
     'created_time': datetime.datetime(2023, 12, 25, 7, 47, 50, 40000, tzinfo=TzInfo(UTC)),
     'deleted_time': None,
     'edgewise_bend_mom_crosstalk_correcte

Notice that when we query we can easily get very large structures. So we should always use query with filtering options to only query the specific part of the data you have an interesst. 

## Querying with Direct Relations

We can include the direct relations by specifing them on the `.query()` call

In [19]:
windmills = client.windmill(limit=2).query(retrieve_nacelle=True, retrieve_rotor=True)

In [21]:
windmills[0].nacelle

,value
external_id,nacelle:1
acc_from_back_side_x,V52-WindTurbine.Acc1N
acc_from_back_side_y,V52-WindTurbine.Acc2N
acc_from_back_side_z,V52-WindTurbine.Acc3N
gearbox,gearbox:1
generator,generator:1
high_speed_shaft,highspeedshaft:1
main_shaft,mainshaft:1
power_inverter,powerinverter:1
yaw_direction,V52-WindTurbine.yaw


Next section: [Creating and Deleting](creating_deleting.html)